In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

geometry_dir = os.path.join("..", "results", "artifacts", "exp13")

target_name = "ShapeResNet50_ShapeResNet"
prefix = "exp13_geometry"

paths = {
    "texture_natural": os.path.join(
        geometry_dir, f"{prefix}_{target_name}_texture_natural.npz"
    ),
    "shape_bias": os.path.join(
        geometry_dir, f"{prefix}_{target_name}_shape_bias.npz"
    ),
}

data_nat = np.load(paths["texture_natural"], allow_pickle=True)
data_shape = np.load(paths["shape_bias"], allow_pickle=True)

list(data_nat.keys())


In [ ]:
from scipy.spatial import ConvexHull

def plot_geometry(data, title):
    coords = data["coords"]
    labels = data["labels"]
    peer_mask = data["peer_mask"]
    hull_vertices = data["hull_vertices"]
    target_idx = int(data["target_idx"])
    mix_idx = int(data["mix_idx"])
    explained = data["explained_variance"]

    peers_coords = coords[peer_mask]
    peers_labels = labels[peer_mask]

    target_coord = coords[target_idx]
    mix_coord = coords[mix_idx]

    hull_pts = peers_coords[hull_vertices]

    fig, ax = plt.subplots(figsize=(5, 5))

    # peer convex hull
    ax.fill(
        hull_pts[:, 0],
        hull_pts[:, 1],
        alpha=0.15,
        edgecolor="C0",
        facecolor="C0",
        label="peer convex hull",
    )

    # peer points
    ax.scatter(peers_coords[:, 0], peers_coords[:, 1], color="C0", s=50)
    for name, (x, y) in zip(peers_labels, peers_coords):
        ax.text(x + 0.01, y, name, fontsize=8, va="center")

    # mix
    ax.scatter(mix_coord[0], mix_coord[1], marker="X", s=80, color="black", label="peer_mix")

    # target
    ax.scatter(
        target_coord[0],
        target_coord[1],
        marker="*",
        s=120,
        color="tab:red",
        label=data["target_name"].item() if hasattr(data["target_name"], "item") else data["target_name"],
    )

    # arrow from mix to target
    ax.arrow(
        mix_coord[0],
        mix_coord[1],
        target_coord[0] - mix_coord[0],
        target_coord[1] - mix_coord[1],
        length_includes_head=True,
        head_width=0.05,
        head_length=0.08,
        alpha=0.7,
        color="tab:red",
    )

    ax.set_xlabel("PCA dim 1")
    ax.set_ylabel("PCA dim 2")
    ev = data["explained_variance"]
    ax.set_title(
        f"{title}\nPCA variance explained: {ev[0]:.2f}, {ev[1]:.2f}"
    )
    ax.grid(True, linestyle="--", alpha=0.3)
    ax.legend(loc="best")
    plt.tight_layout()
    plt.show()


In [ ]:
plot_geometry(data_nat,  "Exp13: geometry (texture_natural)")
plot_geometry(data_shape, "Exp13: geometry (shape_bias)")


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 4.5))

for ax, data, title in zip(
    axes,
    [data_nat, data_shape],
    ["texture_natural", "shape_bias"],
):
    coords = data["coords"]
    labels = data["labels"]
    peer_mask = data["peer_mask"]
    hull_vertices = data["hull_vertices"]
    target_idx = int(data["target_idx"])
    mix_idx = int(data["mix_idx"])

    peers_coords = coords[peer_mask]
    peers_labels = labels[peer_mask]
    target_coord = coords[target_idx]
    mix_coord = coords[mix_idx]
    hull_pts = peers_coords[hull_vertices]

    ax.fill(hull_pts[:, 0], hull_pts[:, 1], alpha=0.15, edgecolor="C0", facecolor="C0")
    ax.scatter(peers_coords[:, 0], peers_coords[:, 1], color="C0", s=40)
    for name, (x, y) in zip(peers_labels, peers_coords):
        ax.text(x + 0.01, y, name, fontsize=7, va="center")

    ax.scatter(mix_coord[0], mix_coord[1], marker="X", s=70, color="black")
    ax.scatter(target_coord[0], target_coord[1], marker="*", s=100, color="tab:red")
    ax.arrow(
        mix_coord[0],
        mix_coord[1],
        target_coord[0] - mix_coord[0],
        target_coord[1] - mix_coord[1],
        length_includes_head=True,
        head_width=0.04,
        head_length=0.06,
        alpha=0.7,
        color="tab:red",
    )

    ax.set_title(title)
    ax.grid(True, linestyle="--", alpha=0.3)
    ax.set_xlabel("PCA dim 1")
axes[0].set_ylabel("PCA dim 2")

fig.suptitle(f"Exp13: convex-hull geometry for target = {target_name}", y=1.03)
plt.tight_layout()
plt.show()
